In [1]:
from behavior_and_time_alignment import beh_and_time_alignment
from session_preprocessing import ephys_opto_preprocessing, session_crosscorr, ephys_opto_crosscorr
from opto_tagging import opto_plotting_session, opto_plotting_unit
from opto_waveforms_preprocessing import opto_wf_preprocessing
from drift_analysis import plot_session_opto_drift
import matplotlib.pyplot as plt
from utils.beh_functions import session_dirs, get_unit_tbl
import numpy as np
import pandas as pd
import os
from utils.plot_utils import combine_pdf_big
import spikeinterface as si
from joblib import Parallel, delayed
import pickle

In [2]:
session_assets = pd.read_csv('/root/capsule/code/data_management/session_assets.csv')
session_list = session_assets['session_id'].tolist()
session_list[-8:-6]

['behavior_784806_2025-06-18_13-39-50', 'behavior_784806_2025-06-20_13-39-16']

In [10]:
session_dir = session_dirs('behavior_791691_2025-06-24_13-21-29')

Selected experiment1 recording1, length:4862.23


In [11]:
# # combine opto pdfs
# data_type = 'curated'
# def process(session, data_type):
#     pdf_dir = session_dirs(session)[f'opto_dir_fig_{data_type}']
#     output_pdf = os.path.join(session_dirs(session)[f'opto_dir_{data_type}'], f'{session}_opto_tagging_png.pdf')
#     if os.path.exists(pdf_dir) and not os.path.exists(output_pdf):
#         print(session)
#         combine_pdf_big(pdf_dir, output_pdf)
# Parallel(n_jobs=4)(delayed(process)(session, data_type) for session in session_list[-14:-3])
# # for session in session_list[-14:-3]:
# #     try:
# #         process(session, data_type)
# #         plt.close('all')
# #     except:
# #         print(f'Failed to process {session}')

In [3]:
# from joblib import Parallel, delayed
# session_list = ['behavior_751181_2025-02-25_12-12-35',
#                 'behavior_751181_2025-02-26_11-51-19']
data_type = 'raw'
def process(session, data_type='raw'):
    plt.close('all')
    session_dir = session_dirs(session)
    print(session_dir[f'curated_dir_{data_type}'])
    if session_dir[f'curated_dir_{data_type}'] is None:
        return None
    print(f'{session} start')
    beh_and_time_alignment(session) # plot behavior, check alignment between behavior, sound, and ephys
    plt.close('all')
    ephys_opto_preprocessing(session, data_type, 'soma') 
    # # redo spiketimes if not aligned, make opto_table (laser time in nidaq); 
    # # generate a pkl file with opto_resp/lat to each condition
    # plt.close('all')
    # ephys_opto_crosscorr(session, data_type)
    # # separate session into opto/beh, cal cross-correlation
    # plt.close('all')
    opto_wf_preprocessing(session, data_type, 'soma', load_sorting_analyzer=True)
    # recal wf based on conditions. (pre/post, power), cal similarity, euclidean distance, correlation with spont
    # pay attention to difference early and late in the session
    
    # opto_plotting_session(session, data_type, 'soma', plot=True, resp_thresh=0.3, lat_thresh=0.02, save=True)    
    # # save as pickle
    # with open(os.path.join(session_dirs(session)[f'opto_dir_{data_type}'], f'{session}_opto_tagging_metrics.pkl'), 'wb') as f:
    #     opto_tagging_df.to_pickle(f)
    
    # if '751004' in session:
    #     plot_session_opto_drift(session, data_type, update_csv=False)
    # else:     
    #     plot_session_opto_drift(session, data_type, update_csv=True)
    print(f'{session} stop')
        
 
# Parallel(n_jobs=6)(delayed(process)(session, data_type = data_type) for session in session_list[-13:])

for session in session_list[12:]:
    print(f'Checking {session}')
    session_dir = session_dirs(session)
    if get_unit_tbl(session, 'raw', summary=False) is None:
        if get_unit_tbl(session, 'curated', summary=False) is None:
            print('No processed unit table found, processing session')
            process(session, data_type='raw') 
    # elif  session_dir['curated_dir_raw'] is not None:
    #     print('Raw')
    #     process(session, data_type='raw')
# process(session_list[-7])
        
 

Checking ecephys_684930_2023-09-28_12-44-15
Single experiment found: experiment1, recording1
There is no nwb file in the curated directory.
Single experiment found: experiment1, recording1
There is no nwb file in the curated directory.
No unit table found for ecephys_684930_2023-09-28_12-44-15 in raw data.
Single experiment found: experiment1, recording1
There is no nwb file in the curated directory.
Checking ecephys_687697_2023-09-15_11-30-06
Single experiment found: experiment1, recording1
There is no nwb file in the curated directory.
Single experiment found: experiment1, recording1
There is no nwb file in the curated directory.
No unit table found for ecephys_687697_2023-09-15_11-30-06 in raw data.
Single experiment found: experiment1, recording1
There is no nwb file in the curated directory.
Checking ecephys_687697_2023-09-15_12-36-06
Single experiment found: experiment1, recording1
There is no nwb file in the curated directory.
Single experiment found: experiment1, recording1
The

AttributeError: 'NoneType' object has no attribute 'units'

In [20]:
get_unit_tbl('ecephys_684930_2023-09-27_10-04-04', 'raw', summary=False)

Single experiment found: experiment1, recording1
There is no nwb file in the curated directory.
No unit table found for ecephys_684930_2023-09-27_10-04-04 in raw data.


In [25]:
rec = si.load('/root/capsule/data/ecephys_713854_2024-03-05_13-31-20_raw_data/ecephys_compressed/experiment1_Record Node 104#Neuropix-PXI-100.ProbeA.zarr')

OSError: [Errno 5] Input/output error

In [10]:
len(os.listdir(os.path.join(session_dir['opto_dir_raw'], 'figures')))

[]

In [12]:
tbl = get_unit_tbl('behavior_716325_2024-05-30_11-33-46', 'raw')

Selected experiment1 recording1, length:3442.18
No unit table found for behavior_716325_2024-05-30_11-33-46 in raw data.


In [14]:
tbl is None

True

In [6]:
session = 'behavior_791691_2025-06-24_13-21-29'
process(session)
# session_dir = session_dirs(session)

NameError: name 'process' is not defined

In [8]:
session_dir['nwb_dir_raw']

'/root/capsule/data/behavior_791691_2025-06-24_13-21-29_sorted/nwb/behavior_791691_2025-06-24_13-21-26_experiment2_recording1.nwb'

In [4]:
session = 'behavior_791691_2025-06-24_13-21-29'
session_dir = session_dirs(session)

Selected experiment1 recording1, length:4862.23


In [4]:
session_dir['curated_dir_raw']

'/root/capsule/data/behavior_782394_2025-04-22_10-53-28_sorted/curated/experiment1_Record Node 104#Neuropix-PXI-100.ProbeA_recording1'

In [ ]:
os.path.exists(session_dir['sorted_dir_raw'])

True

In [4]:
from open_ephys.analysis import Session

In [5]:
rec_file = '/root/capsule/data/behavior_791691_2025-06-24_13-21-29_raw_data/ecephys/ecephys_clipped'
rec_clipped = Session(rec_file)

In [29]:
stream_info = get_stream_info("/root/capsule/data/ecephys_713854_2024-03-05_13-31-20_raw_data/ecephys_clipped")

In [30]:
stream_info

,Record Node,Rec Idx,Exp Idx,Stream,Duration (s),Channels
0,104,0,0,ProbeA,0.003333,384
1,104,0,0,PXIe-6341,0.003333,8


In [19]:
rec_clipped.recordnodes[0].recordings[0].continuous[0].timestamps[-1]-rec_clipped.recordnodes[0].recordings[0].continuous[0].timestamps[0]

np.float64(5118.98209682107)

In [22]:
rec_clipped.recordnodes[0]

AttributeError: 'RecordNode' object has no attribute 'metadata'

In [ ]:
# make a df from all_Tm, columns correspond to focus regressors
 